In [14]:
###############################################
# Rebrov reactor batch script
# Ting-Chen Lee and Chris Blais
# Northeastern University
# runs through all reactor conditions
###############################################

import numpy as np
import time
import cantera as ct
from matplotlib import pyplot as plt
import csv
import math
import os
import sys
import re
import itertools
import logging
from collections import defaultdict
import git
import json

from rmgpy.molecule import Molecule
from rmgpy.data.base import Database

def save_pictures(git_path="", species_path="", overwrite=False):
    """
    Save a folder full of molecule pictures, needed for the pretty dot files.

    Saves them in the results directory, in a subfolder "species_pictures".
    Unless you set overwrite=True, it'll leave alone files that are
    already there.
    """
    dictionary_filename = git_path + "/base/chemkin/species_dictionary.txt"
    specs = Database().get_species(dictionary_filename, resonance=False)

    images_dir = os.path.join(species_path)
    os.makedirs(images_dir, exist_ok=True)
    for name, species in specs.items():
        filepath = os.path.join(images_dir, name + ".png")
        if not overwrite and os.path.exists(filepath):
            continue
        species.molecule[0].draw(filepath)

def prettydot(species_path, dotfilepath, strip_line_labels=False):
    """
    Make a prettier version of the dot file (flux diagram)

    Assumes the species pictures are stored in a directory
    called 'species_pictures' alongside the dot file.
    """
    
    pictures_directory = f"{species_path}/"

    if strip_line_labels:
        print("stripping edge (line) labels")

    reSize = re.compile('size="5,6"\;page="5,6"')
    reNode = re.compile(
        '(?P<node>s\d+)\ \[\ fontname="Helvetica",\ label="(?P<label>[^"]*)"\]\;'
    )

    rePicture = re.compile("(?P<smiles>.+?)\((?P<id>\d+)\)\.png")
    reLabel = re.compile("(?P<name>.+?)\((?P<id>\d+)\)$")

    species_pictures = dict()
    for picturefile in os.listdir(pictures_directory):
        match = rePicture.match(picturefile)
        if match:
            species_pictures[match.group("id")] = picturefile
        else:
            pass
            # print(picturefile, "didn't look like a picture")

    filepath = dotfilepath

    if not open(filepath).readline().startswith("digraph"):
        raise ValueError("{0} - not a digraph".format(filepath))

    infile = open(filepath)
    prettypath = filepath.replace(".dot", "", 1) + "-pretty.dot"
    outfile = open(prettypath, "w")

    for line in infile:
        (line, changed_size) = reSize.subn('size="12,12";page="12,12"', line)
        match = reNode.search(line)
        if match:
            label = match.group("label")
            idmatch = reLabel.match(label)
            if idmatch:
                idnumber = idmatch.group("id")
                if idnumber in species_pictures:
                    line = (
                        f'%s [ image="{pictures_directory}%s" label="" width="0.5" height="0.5" imagescale=false fixedsize=false color="none" ];\n'
                        % (match.group("node"), species_pictures[idnumber])
                    )

        # rankdir="LR" to make graph go left>right instead of top>bottom
        if strip_line_labels:
            line = re.sub('label\s*=\s*"\s*[\d.]+"', 'label=""', line)

        # change colours
        line = re.sub('color="0.7,\ (.*?),\ 0.9"', r'color="1.0, \1, 0.7*\1"', line)

        outfile.write(line)

    outfile.close()
    infile.close()
#     print(f"Graph saved to: {prettypath}")
    os.system(f'dot {prettypath} -Tpng -o{prettypath.replace(".dot", "", 1) + ".png"} -Gdpi=200')
    return prettypath

def show_flux_diagrams(self, suffix="", embed=False):
    """
    Shows the flux diagrams in the notebook.
    Loads them from disk.
    Does not embed them, to keep the .ipynb file small,
    unless embed=True. Use embed=True if you might over-write the files,
    eg. you want to show flux at different points.
    """
    import IPython

    for element in "CHON":
        for phase_object in (self.gas, self.surf):
            phase = phase_object.name
            img_file = (
                f"reaction_path_{element}_{phase}{'_' if suffix else ''}{suffix}.png"
            )
            display(IPython.display.HTML(f"<hr><h2>{element} {phase}</h2>"))
            if embed:
                display(IPython.display.Image(filename=img_file, width=400, embed=True))
            else:
                display(IPython.display.Image(url=img_file, width=400, embed=False))

        # Now do the combined
        img_file = f"reaction_path_mass{'_' if suffix else ''}{suffix}.png"
        display(IPython.display.HTML(f"<hr><h2>Combined mass</h2>"))
        if embed:
            display(IPython.display.Image(filename=img_file, width=400, embed=True))
        else:
            display(IPython.display.Image(url=img_file, width=400, embed=False))

def save_flux_diagrams(*phases, suffix="", timepoint="", species_path=""):
    """
    Saves the flux diagrams. The filenames have a suffix if provided,
    so you can keep them separate and not over-write.
    """
    for element in "CHON":
        for phase_object in phases:
            phase = phase_object.name

            diagram = ct.ReactionPathDiagram(phase_object, element)
            diagram.title = f"Reaction path diagram following {element} in {phase}"
            diagram.label_threshold = 0.001

            dot_file = f"{suffix}/reaction_path_{element}_{phase}_{timepoint}.dot"
            img_file = f"{suffix}/reaction_path_{element}_{phase}_{timepoint}.png"
            dot_bin_path = (
                "/Users/lee.ting/Code/anaconda3/pkgs/graphviz-2.40.1-hefbbd9a_2/bin/dot" 
                #maybe try "/home/lee.ting/.conda/pkgs/graphviz-2.40.1-h21bd128_2/bin/dot"
            )
            img_path = os.path.join(os.getcwd(), img_file)
            diagram.write_dot(dot_file)

            #also make a prettydot file
            prettydot(species_path, dot_file, strip_line_labels=False)

            # print(diagram.get_data())

#             print(
#                 f"Wrote graphviz input file to '{os.path.join(os.getcwd(), dot_file)}'."
#             )
            os.system(f"dot {dot_file} -Tpng -o{img_file} -Gdpi=200")
#             print(f"Wrote graphviz output file to '{img_path}'.")






In [2]:
# #atol_rtol_array = [1e-9, 1e-10,1e-11,1e-12, 1e-13,1e-14,1e-15,1e-16,1e-17,
# #                   1e-18,1e-19,1e-20,1e-21,1e-22,1e-23,1e-24,1e-25,1e-26,1e-27]
# #rtol_diff = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7,1e-8, 1e-9, 1e-10]
# atol_rtol_array = np.logspace(-29, -9, num=21)
# rtol_diff = np.logspace(1, 10, num=10)


In [15]:
#######################################################################
# Input Parameters for combustor
#######################################################################

# filepath for writing files
notebook_dir = '/work/westgroup/lee.ting/cantera/ammonia_analysis/Troubleshooting/'
git_repo = "/work/westgroup/lee.ting/cantera/ammonia/"
cti_file = git_repo + "base/cantera/chem_annotated.cti"

temp = 589 # kelvin
pressure = 1 * ct.one_atm  # Pascals
volume_flow = 5.8333e-5 # [m^3/s]
#3500 Ncm3/min = 3500/e6/60 m3/s = 5.8333e-5

# NH3/O2 = 0.068
X_o2 = 0.15379310344827588
x_O2_str = str(X_o2)[0:3].replace(".", "_")
X_nh3 = 0.066  
x_NH3_str = str(X_nh3)[0:8].replace(".", "_")
X_he = 1 - X_o2 - X_nh3

# sensitivity settings
sensitivity = False
sensatol = 1e-6
sensrtol = 1e-6

# Specify idealGasReactor and isothermal
reactor_type=1
energy="off"

# 14 aluminum plates, each of them containing seven semi-cylindrical microchannels of 280 µm width 
# and 140 µm depth, 9 mm long, arranged at equal distances of 280 µm 

# get git commit hash and message
rmg_model_path = "../../ammonia"
repo = git.Repo(rmg_model_path)
date = time.localtime(repo.head.commit.committed_date)
git_date = f"{date[0]}_{date[1]}_{date[2]}_{date[3]}{date[4]}"
git_sha = str(repo.head.commit)[0:6]
git_msg = str(repo.head.commit.message)[0:50].replace(" ", "_").replace("'", "_").replace("\n", "")
git_file_string = f"{git_date}_{git_sha}_{git_msg}"

# set sensitivity string for file path name
if sensitivity:
    sensitivity_str = "on"
else: 
    sensitivity_str = "off"


# constants
pi = math.pi

surf_temp = temp

mw_nh3 = 17.0306e-3  # [kg/mol]
mw_o2 = 31.999e-3  # [kg/mol]
mw_he = 4.002602e-3  # [kg/mol]

o2_ratio = X_nh3 / X_o2

# O2/NH3/He: typical is
concentrations_rmg = {"O2(2)": X_o2, "NH3(6)": X_nh3, "He": X_he}

# initialize cantera gas and surface
gas = ct.Solution(cti_file, "gas")
surf = ct.Interface(cti_file, "surface1", [gas])

# initialize temperatures 
gas.TPX = temp, pressure, concentrations_rmg
surf.TP = temp, pressure # change this to surf_temp when we want a different starting temperature for the surface

# if a mistake is made with the input, 
# cantera will normalize the mole fractions. 
# make sure that we are reporting/using 
# the normalized values
X_o2 = float(gas["O2(2)"].X)
X_nh3 = float(gas["NH3(6)"].X)
X_he = float(gas["He"].X)

# create gas inlet
inlet = ct.Reservoir(gas)

# create gas outlet
exhaust = ct.Reservoir(gas)

# Reactor volume
number_of_reactors = 1001
rradius = 1.4e-4 #140µm to 0.00014m
rtotal_length = 9e-3 #9mm to 0.009m
rtotal_vol = (rradius ** 2) * pi * rtotal_length / 2

rlength = rtotal_length/(number_of_reactors-1)

# divide totareactor total volume 
rvol = (rtotal_vol )/number_of_reactors

# Catalyst Surface Area
site_density = (surf.site_density * 1000)  # [mol/m^2] cantera uses kmol/m^2, convert to mol/m^2
cat_area_total = rradius * 2 * pi * rtotal_length # [m^3] /2 for semi-cylinder
cat_area = cat_area_total / number_of_reactors

# reactor initialization
if reactor_type == 0:
    r = ct.Reactor(gas, energy=energy)
    reactor_type_str = "Reactor"
elif reactor_type == 1:
    r = ct.IdealGasReactor(gas, energy=energy)
    reactor_type_str = "IdealGasReactor"
elif reactor_type == 2:
    r = ct.ConstPressureReactor(gas, energy=energy)
    reactor_type_str = "ConstPressureReactor"
elif reactor_type == 3:
    r = ct.IdealGasConstPressureReactor(gas, energy=energy)
    reactor_type_str = "IdealGasConstPressureReactor"

# calculate the available catalyst area in a differential reactor
rsurf = ct.ReactorSurface(surf, r, A=cat_area)
r.volume = rvol
surf.coverages = "X(1):1.0"

# flow controllers 
one_atm = ct.one_atm
FC_temp = 298.15

molar_flow = volume_flow * one_atm / (8.3145 * FC_temp)  # [mol/s]
mass_flow = molar_flow * (X_nh3 * mw_nh3 + X_o2 * mw_o2 + X_he * mw_he)  # [kg/s]
mfc = ct.MassFlowController(inlet, r, mdot=mass_flow)

# A PressureController has a baseline mass flow rate matching the 'master'
# MassFlowController, with an additional pressure-dependent term. By explicitly
# including the upstream mass flow rate, the pressure is kept constant without
# needing to use a large value for 'K', which can introduce undesired stiffness.
outlet_mfc = ct.PressureController(r, exhaust, master=mfc, K=0.01)

# initialize reactor network
sim = ct.ReactorNet([r])

# set relative and absolute tolerances on the simulation
sim.rtol = 1e-11
sim.atol = 1e-22

#surf.advance_coverages(100)

#################################################
# Run single reactor
#################################################

# round numbers for filepath strings so they're easier to read
cat_area_str = "%s" % "%.3g" % cat_area

# if it doesn't already exist, create

results_path = (
    notebook_dir 
    + f"/results"
)
# create species folder for results if they don't exist already

try:
    os.makedirs(results_path, exist_ok=True)
except OSError as error:
    print(error)

gas_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.species_names]

# surface ROP reports gas and surface ROP. these values are not redundant
gas_surf_ROP_str = [i + " surface ROP [kmol/m^2 s]" for i in gas.species_names]
surf_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.species_names]

# gasrxn_ROP_str = [i + " ROP [kmol/m^3 s]" for i in gas.reaction_equations()]
surfrxn_ROP_str = [i + " ROP [kmol/m^2 s]" for i in surf.reaction_equations()]

output_filename = (
    results_path
    + f"/Spinning_basket_results.csv"
)

outfile = open(output_filename, "w")
writer = csv.writer(outfile)

# Sensitivity atol, rtol, and strings for gas and surface reactions if selected
# slows down script by a lot
if sensitivity:
    sim.rtol_sensitivity = sensrtol
    sim.atol_sensitivity = sensatol
    sens_species = ["NH3(6)", "O2(2)", "N2(4)", "NO(5)", "N2O(7)"]  #change THIS to your species, can add "," and other species

    # turn on sensitive reactions
    for i in range(gas.n_reactions):
        r.add_sensitivity_reaction(i)

    for i in range(surf.n_reactions):
        rsurf.add_sensitivity_reaction(i)
    
    # create column names for sensitivity data
    for j in sens_species:
        gasrxn_sens_str = [
            j + " sensitivity to " + i for i in gas.reaction_equations()
        ]
        surfrxn_sens_str = [
            j + " sensitivity to " + i for i in surf.reaction_equations()
        ]
        sens_list = gasrxn_sens_str + surfrxn_sens_str  # + gastherm_sens_str

    writer.writerow(
        [
            "Distance (mm)",
            "T (C)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial", 
            "X_o2 initial",
            "X_he initial",
            "(NH3/O2)",
            "T (C) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
#         + gasrxn_ROP_str
        + surfrxn_ROP_str
        + sens_list
    )

else:

    writer.writerow(
        [
            "Distance (mm)",
            "T (C)",
            "P (Pa)",
            "V (M^3/s)",
            "X_nh3 initial",
            "X_o2 initial",
            "X_he initial",
            "(NH3/O2)",
            "T (C) final",
            "Rtol",
            "Atol",
            "reactor type",
            "energy on?"
        ]
        + gas.species_names
        + surf.species_names
        + gas_ROP_str
        + gas_surf_ROP_str
        + surf_ROP_str
#         + gasrxn_ROP_str
        + surfrxn_ROP_str
    )

# initialize time and iteration count, set time step
t = 0.0
dt = 0.1
iter_ct = 0

# run the simulation
first_run = True
distance_mm = 0

for n in range(number_of_reactors):


    # Set the state of the reservoir to match that of the previous reactor
    gas.TDY = TDY = r.thermo.TDY
    inlet.syncState()
    sim.reinitialize()
    previous_coverages = surf.coverages  # in case we want to retry
    
    # Add a first row in the CSV with just the feed
    writer.writerow(
        [
            distance_mm,
            temp,
            gas.P,
            volume_flow,
            X_nh3,
            X_o2,
            X_he,
            o2_ratio,
            gas.T,
            sim.rtol,
            sim.atol,
            reactor_type_str,
            energy,
        ]
        + list(gas.X)
        + list(surf.X)
        + list(gas.net_production_rates)
        + list(surf.net_production_rates)
#         + list(gas.net_rates_of_progress)
        + list(surf.net_rates_of_progress)
    )
    if n > 0:  
        try:
            sim.advance_to_steady_state(return_residuals=True)
        except ct.CanteraError:
            t = sim.time
            sim.set_initial_time(0)
            gas.TDY = TDY
            surf.coverages = previous_coverages
            r.syncState()
            sim.reinitialize()
            new_target_time = 0.01 * t
            logging.warning(
                f"Couldn't reach {t:.1g} s so going to try {new_target_time:.1g} s"
            )
            try:
                sim.advance(new_target_time)
            except ct.CanteraError:
                outfile.close()
                raise

    if sensitivity:
        # get sensitivity for sensitive species i (e.g. methanol) in reaction j
        for i in sens_species:
            g_nrxn = gas.n_reactions
            s_nrxn = surf.n_reactions

            gas_sensitivities = [sim.sensitivity(i, j) for j in range(g_nrxn)]
            surf_sensitivities = [
                sim.sensitivity(i, j) for j in range(g_nrxn, g_nrxn + s_nrxn)
            ]

            sensitivities_all = (
                gas_sensitivities
                + surf_sensitivities
            )

        writer.writerow(
            [
                distance_mm,
                temp,
                gas.P,
                volume_flow,
                X_nh3,
                X_o2,
                X_he,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
#             + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
            + sensitivities_all,
        )

    else:
        writer.writerow(
            [
                distance_mm,
                temp,
                gas.P,
                volume_flow,
                X_nh3,
                X_o2,
                X_he,
                o2_ratio,
                gas.T,
                sim.rtol,
                sim.atol,
                reactor_type_str,
                energy,
            ]
            + list(gas.X)
            + list(surf.X)
            + list(gas.net_production_rates)
            + list(surf.net_production_rates)
#             + list(gas.net_rates_of_progress)
            + list(surf.net_rates_of_progress)
        )


    iter_ct += 1
    distance_mm = n * rlength * 1.0e3  # distance in mm

outfile.close()

#print(f"passed! atol={atol_rtol_array}")




H2OX <=> H2O + X

In [24]:
# reaction_eq = surf.reaction_equation()
# print(reaction_eq)
# surf.set_multiplier(0,1)
# print(surf.multiplier(1))

In [20]:
type(surf.forward_rate_constants)

numpy.ndarray

In [21]:
for i,j in enumerate(surf.reaction_equations()):
    if "N2(4)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

2 NX <=> N2(4) + 2 X(1) rate constant:  3.24e+09
N2O(7) + X(1) <=> N2(4) + OX rate constant:  3.40e+01


In [35]:
for i,j in enumerate(surf.reaction_equations()):
    if "NO(5)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

NO_X(14) <=> NO(5) + X(1) rate constant:  6.25e+00


In [36]:
for i,j in enumerate(surf.reaction_equations()):
    if "N2O(7)" in j:
        print(j, "rate constant: ", "{:.2e}".format(surf.forward_rate_constants[i]))

NO_X(14) + N_X(13) <=> N2O(7) + 2 X(1) rate constant:  3.42e+09
N2O(7) + X(1) <=> N2(4) + O_X(9) rate constant:  6.80e+01
N2O_X(51) <=> N2O(7) + X(1) rate constant:  2.80e+16
N2O(7) + X(1) <=> N2O_X(48) rate constant:  1.07e+10


In [25]:
surf_rates = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_rates[j]=surf.forward_rate_constants[i]
    
{k: v for k, v in sorted(surf_rates.items(), key=lambda item: item[1], reverse=True)}


{'NH2_X(8) + OX <=> NH_X(9) + OHX': 4.03e+20,
 'NH_X(9) + OHX <=> H2O_X(11) + NX': 3.3576604796026396e+16,
 'NH_X(9) + OX <=> NX + OHX': 2614294111925253.0,
 'H2O_X(11) + OX <=> 2 OHX': 85432429410677.88,
 'NH3(6) + X(1) <=> NH3X': 6869345693.615923,
 '2 NX <=> N2(4) + 2 X(1)': 3244863665.0109177,
 'H2O_X(11) <=> H2O(3) + X(1)': 2667441879.931186,
 'NX + OX <=> NOX + X(1)': 2041515732.8237882,
 'NOX + NX <=> N2O(7) + 2 X(1)': 1712224310.2844157,
 'NH3X + OX <=> NH2_X(8) + OHX': 817710493.8685722,
 'N2O(7) + X(1) <=> N2(4) + OX': 33.985988615815934,
 'NOX <=> NO(5) + X(1)': 3.122602390824976,
 'O2(2) + 2 X(1) <=> 2 OX': 0.0}

In [38]:
gas_rates = {}
for i,j in enumerate(gas.reaction_equations()):
    gas_rates[j]=gas.forward_rate_constants[i]
{k: v for k, v in sorted(gas_rates.items(), key=lambda item: item[1], reverse=True)}


{}

In [39]:
gas_equilib = {}
for i,j in enumerate(gas.reaction_equations()):
    gas_equilib[j]=gas.equilibrium_constants[i]
{k: v for k, v in sorted(gas_equilib.items(), key=lambda item: item[1], reverse=True)}


{}

In [40]:
surf_equilib = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_equilib[j]=surf.equilibrium_constants[i]
{k: v for k, v in sorted(surf_equilib.items(), key=lambda item: item[1], reverse=True)}


{'N2O(7) + X(1) <=> N2(4) + O_X(9)': 1.2991255597245907e+19,
 'HONO(71) + HX(18) <=> H2O_X(16) + NO_X(14)': 3.6572720609315565e+17,
 'O2(2) + 2 X(1) <=> 2 O_X(9)': 2.912928870321356e+17,
 '2 N_X(13) <=> N2(4) + 2 X(1)': 211969311760492.0,
 'NHO_X(15) + OH_X(11) <=> H2O_X(16) + NO_X(14)': 161967321566954.06,
 'HO2X(60) + X(1) <=> OH_X(11) + O_X(9)': 68039917623694.98,
 'HO2X(60) + N_X(13) <=> HNOX(69) + O_X(9)': 37244811518.80247,
 'NH2_X(10) + NHO_X(15) <=> NH3_X(8) + NO_X(14)': 25403200375.620384,
 'HX(18) + OH_X(11) <=> H2O_X(16) + X(1)': 6945859732.173205,
 'NH2_X(10) + OH_X(11) <=> H2O_X(16) + NH_X(12)': 4243263603.352928,
 'HONO(70) + 2 X(1) <=> NO_X(14) + OH_X(11)': 1650641341.0251546,
 'NH_X(12) + OH_X(11) <=> H2O_X(16) + N_X(13)': 92595258.31213702,
 'HONO(71) + OH_X(11) <=> H2O_X(16) + NO2_X(28)': 72117118.52908012,
 'HONO(71) + X(1) <=> NO_X(14) + OH_X(11)': 52653986.71947109,
 'NO2(29) + X(1) <=> NO2_X(28)': 18951838.436251506,
 'NHO_X(15) + NO2_X(28) <=> HONO(71) + NO_X(14)

In [10]:
surf_rates = {}
for i,j in enumerate(surf.reaction_equations()):
    surf_rates[j]=surf.reverse_rate_constants[i]
{k: v for k, v in sorted(surf_rates.items(), key=lambda item: item[1], reverse=True)}


{'N2O_X(16) + X(1) <=> N2_X(17) + O_X(9)': 1.8161058802100163e+67,
 'H2O_X(18) + O_X(9) <=> 2 OH_X(11)': 3.9235164275294686e+29,
 'N2O(7) + 2 X(1) <=> N2_X(17) + O_X(9)': 1.6592874489920146e+24,
 'NO_X(14) + N_X(13) <=> N2O(7) + 2 X(1)': 1.1312442779788327e+24,
 'NH2_X(10) + O_X(9) <=> NH_X(12) + OH_X(11)': 3.651083903633921e+22,
 'NO_X(14) <=> NO(5) + X(1)': 1.347203226933607e+22,
 'H2O(3) + 2 X(1) <=> HX(20) + OH_X(11)': 1.0834184109496732e+22,
 'NH_X(12) + O_X(9) <=> NHO_X(15) + X(1)': 2.966828274622335e+21,
 'N2OXX(54) <=> N2_X(17) + O_X(9)': 6.942272090088288e+20,
 'H2O_X(18) + N_X(13) <=> HNOX(61) + HX(20)': 2.4245966473088798e+20,
 'H2O_X(18) + O_X(9) <=> HO2X(50) + HX(20)': 1.3201579712106604e+20,
 'HX(20) + O_X(9) <=> OH_X(11) + X(1)': 1.661326688560399e+19,
 'HONO(64) + O_X(9) <=> HO2X(50) + NO_X(14)': 1.1309877809646213e+19,
 'NO_X(14) + O_X(9) <=> NO2JX(53) + X(1)': 1.0378281422409816e+19,
 'NO_X(14) + N_X(13) <=> N2O_X(16) + X(1)': 7.602876327474689e+18,
 'NHO_X(15) + O_X(

In [41]:
gas_rates = {}
for i,j in enumerate(gas.reaction_equations()):
    gas_rates[j]=gas.reverse_rate_constants[i]
{k: v for k, v in sorted(gas_rates.items(), key=lambda item: item[1], reverse=True)}


{}

In [42]:
sim.component_name(14)


'IdealGasReactor_4: X(1)'

In [45]:
import pandas as pd

In [46]:
pd.read_csv('/work/westgroup/lee.ting/cantera/ammonia_analysis/results/O2_results/2021_6_15_1323_df9fd9_master_500_1000K_O88N066/IdealGasReactor/energy_off/sensitivity_off/598/results/Spinning_basket_area_1.11e-07_energy_off_temp_598_O2_0_15379310344827588_NH3_0.csv')

,Distance (mm),T (K),P (Pa),V (M^3/s),X_nh3 initial,X_o2 initial,X_he initial,(NH3/O2),T (K) final,Rtol,...,HONO(71) + O_X(9) <=> HO2X(60) + NO_X(14) ROP [kmol/m^2 s],HO2X(60) + X(1) <=> OH_X(11) + O_X(9) ROP [kmol/m^2 s].1,N2(4) + X(1) <=> NN_ads(80) ROP [kmol/m^2 s],NO2(29) + X(1) <=> NO2_X(28) ROP [kmol/m^2 s],HNOX(69) + X(1) <=> N_X(13) + OH_X(11) ROP [kmol/m^2 s],H2O_X(16) + N_X(13) <=> HNOX(69) + HX(18) ROP [kmol/m^2 s],HONO(71) + N_X(13) <=> HNOX(69) + NO_X(14) ROP [kmol/m^2 s],HONO(71) + X(1) <=> HNOX(69) + O_X(9) ROP [kmol/m^2 s],HO2X(60) + N_X(13) <=> HNOX(69) + O_X(9) ROP [kmol/m^2 s],HNOX(69) + X(1) <=> N_X(13) + OH_X(11) ROP [kmol/m^2 s].1
0,0.000,598,101325.000000,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,0.009,598,101323.974490,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,2.034707e-21,2.824363e-26,8.833756e-16,-4.477294e-26,-9.717243e-17,1.199820e-16,3.428533e-28,-1.211509e-17,5.410720e-23,-9.717243e-17
2,0.018,598,101324.928238,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,3.072798e-21,8.452669e-26,1.199449e-15,-7.886949e-26,-1.206005e-16,4.729479e-16,9.071140e-28,-2.501318e-17,1.508850e-22,-1.206005e-16
3,0.027,598,101324.916637,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,3.824771e-21,1.377384e-25,1.482128e-15,-1.057804e-25,-1.313025e-16,5.299491e-16,1.534528e-27,-3.729557e-17,2.611903e-22,-1.313025e-16
4,0.036,598,101324.718323,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,4.451638e-21,1.879203e-25,1.759045e-15,-1.296341e-25,-1.379684e-16,5.475775e-16,2.206941e-27,-4.915465e-17,3.808332e-22,-1.379684e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.567,598,101316.545595,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,-6.092251e-28,-1.416773e-30,1.068383e-23,-2.019306e-22,-1.729667e-28,3.385212e-30,4.129478e-39,-3.471474e-28,5.465712e-30,-1.729667e-28
64,0.576,598,101316.894619,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,-1.160801e-28,-1.417122e-30,1.140654e-23,-2.019314e-22,-1.730085e-28,3.385237e-30,4.130508e-39,-3.492870e-28,5.465621e-30,-1.730085e-28
65,0.585,598,101319.673208,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,2.298585e-29,-1.417238e-30,1.174916e-23,-2.019316e-22,-1.730224e-28,3.385245e-30,4.130817e-39,-3.500113e-28,5.465620e-30,-1.730224e-28
66,0.594,598,101320.759178,2.771000e-08,0.066,0.153793,0.780207,0.429148,598.0,1.000000e-11,...,1.399578e-29,-1.417078e-30,1.292011e-23,-2.019314e-22,-1.730030e-28,3.385232e-30,4.130728e-39,-3.503035e-28,5.465527e-30,-1.730030e-28
